## Calculate national-level intersects of butterfly data

* Read in the butterfly data
* Read in national shapes
* Count intersecting points for a given national shape
* Create a simple data table, export as csv

In [1]:
import pandas as pd
import geopandas as gpd
import shapely